# Notebook 02: Heston Model and Analytical Pricing Methods

This notebook demonstrates the Heston stochastic volatility model and various analytical pricing methods for European options. We'll cover:

1. **Heston Model Implementation** - Stochastic volatility model with characteristic function
2. **COS Method** - Fourier-Cosine series expansion for option pricing
3. **Fourier Transform Method** - Fast Fourier Transform approach
4. **Monte Carlo Simulation** - Numerical simulation for pricing
5. **Black-Scholes Comparison** - Traditional constant volatility model
6. **Performance Analysis** - Speed and accuracy comparisons

## Heston Model

The Heston model describes the evolution of asset price $S_t$ and variance $v_t$:

$$dS_t = rS_t dt + \sqrt{v_t}S_t dW_1^t$$

$$dv_t = \kappa(\theta - v_t)dt + \sigma\sqrt{v_t}dW_2^t$$

Where:
- $r$: Risk-free rate
- $\kappa$: Mean reversion speed
- $\theta$: Long-term variance
- $\sigma$: Volatility of variance
- $\rho$: Correlation between $W_1^t$ and $W_2^t$


In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

# Import our pricing implementations
import sys
sys.path.append('../src')

from elm.models.pricing.models.heston import HestonModel
from elm.models.pricing.methods.cos import COSPricer
from elm.models.pricing.methods.fourier import FourierPricer
from elm.models.pricing.methods.monte_carlo import MonteCarlo
from elm.models.pricing.methods.black_scholes import BlackScholes

# Set random seed for reproducibility
np.random.seed(42)

print("   - HestonModel: Stochastic volatility model")
print("   - COSPricer: Fourier-Cosine method")
print("   - FourierPricer: Fast Fourier Transform")
print("   - MonteCarlo: Monte Carlo simulation")
print("   - BlackScholes: Traditional constant volatility model")


   - HestonModel: Stochastic volatility model
   - COSPricer: Fourier-Cosine method
   - FourierPricer: Fast Fourier Transform
   - MonteCarlo: Monte Carlo simulation
   - BlackScholes: Traditional constant volatility model


In [3]:
# 1. Heston Model Implementation

# Define typical market parameters
S0 = 100.0      # Initial stock price
K = 105.0       # Strike price
T = 1.0         # Time to maturity (1 year)
r = 0.05        # Risk-free rate
q = 0.02        # Dividend yield

# Heston model parameters
v0 = 0.04       # Initial variance (20% vol)
theta = 0.04    # Long-term variance (20% vol)
kappa = 2.0     # Mean reversion speed
sigma = 0.3     # Volatility of variance
rho = -0.7      # Correlation

print(f"   Spot Price (S0): {S0}")
print(f"   Strike Price (K): {K}")
print(f"   Time to Maturity (T): {T}")
print(f"   Risk-free Rate (r): {r}")
print(f"   Dividend Yield (q): {q}")

print(f"   Initial Variance (v0): {v0}")
print(f"   Long-term Variance (θ): {theta}")
print(f"   Mean Reversion (κ): {kappa}")
print(f"   Vol of Vol (σ): {sigma}")
print(f"   Correlation (ρ): {rho}")

# Create Heston model
heston_model = HestonModel(
    S0=S0, r=r, q=q, v0=v0, theta=theta, 
    kappa=kappa, sigma=sigma, rho=rho
)

print(f"   Characteristic function available: {hasattr(heston_model, 'characteristic_function')}")

# Test characteristic function
u_test = 1.0 + 0.5j
cf_value = heston_model.characteristic_function(u_test, T)
print(f"   Characteristic function test: φ({u_test}, {T}) = {cf_value:.6f}")


   Spot Price (S0): 100.0
   Strike Price (K): 105.0
   Time to Maturity (T): 1.0
   Risk-free Rate (r): 0.05
   Dividend Yield (q): 0.02
   Initial Variance (v0): 0.04
   Long-term Variance (θ): 0.04
   Mean Reversion (κ): 2.0
   Vol of Vol (σ): 0.3
   Correlation (ρ): -0.7
   Characteristic function available: True
   Characteristic function test: φ((1+0.5j), 1.0) = -0.011500-0.097066j


In [12]:
# 2. COS Method Pricing

# Create COS pricer
cos_pricer = COSPricer(N=256, L=10.0)

# Price call option using COS method
start_time = time.time()
cos_price = cos_pricer.price(
    cf=heston_model.characteristic_function,
    K=K, T=T, r=r,
    option_type="call"
)
cos_time = time.time() - start_time


print(f"   Call Option Price: {cos_price:.4f}")
print(f"   Computation Time: {cos_time*1000:.2f} ms")

# Price put option
put_price = cos_pricer.price(
    cf=heston_model.characteristic_function,
    K=K, T=T, r=r,
    option_type="put"
)

print(f"   Put Option Price: {put_price:.4f}")

# Test put-call parity
forward_price = S0 * np.exp(-q * T) - K * np.exp(-r * T)
put_call_parity = cos_price - put_price
print(f"   Put-Call Parity Check: {put_call_parity:.6f} (Expected: {forward_price:.6f})")

# Test different COS parameters
for N in [64, 128, 256, 512]:
    cos_test = COSPricer(N=N, L=10.0)
    price_test = cos_pricer.price(
    cf=heston_model.characteristic_function,
    K=K, T=T, r=r,
    option_type="call"
)
    print(f"   N={N:3d}: Price = {price_test:.6f}")


   Call Option Price: 6.5348
   Computation Time: 1.27 ms
   Put Option Price: 8.3940
   Put-Call Parity Check: -1.859222 (Expected: -1.859222)
   N= 64: Price = 6.534791
   N=128: Price = 6.534791
   N=256: Price = 6.534791
   N=512: Price = 6.534791
